In [2]:
# imports
import os
import requests
from bs4 import BeautifulSoup
from typing import List
import gradio as gr
import ollama

In [8]:
system_message = "You are a helpful assistant"
MODEL="llama3.2:latest"

In [9]:
def message_llama(propmpt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": propmpt}
    ]
    completion = ollama.chat(model=MODEL, messages=messages)
    return completion.message.content

In [10]:
message_llama("What is today's weather?")

"I'm not capable of providing real-time weather information. However, I can suggest some ways for you to find out the current weather:\n\n1. Check online weather websites such as AccuWeather, Weather.com, or the National Weather Service (NWS).\n2. Use a mobile app like Dark Sky, Weather Underground, or The Weather Channel.\n3. Tune into local news or check their website for weather updates.\n\nIf you'd like to know the typical weather patterns for a specific location, feel free to provide me with the city or region, and I'll do my best to provide general information!"

In [11]:
# here's a simple function
def shout(text):
    print(f"Shout has been called with input: {text}")
    return text.upper()

In [12]:
shout("hello")

Shout has been called with input: hello


'HELLO'

In [ ]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox").launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Shout has been called with input: hehe
Shout has been called with input: hehe
Shout has been called with input: hehe
Created dataset file at: .gradio\flagged\dataset1.csv


In [14]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True)


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [15]:
# Inputs and Outputs
view = gr.Interface(
    fn=shout,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=6)],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


Shout has been called with input: how 


In [19]:
system_message = "You are a helpful assistant that responds in markdown"

view = gr.Interface(
    fn=message_llama,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never",
    title="Llama Chat",
    description="A simple chatbot that responds in markdown."
)

view.launch()

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


In [20]:
# Let's create a call that streams back results

def stream_llama(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]
    completion = ollama.chat(model=MODEL, messages=messages, stream=True)
    result = ""
    for message in completion:
        result += message.message.content
        yield result

In [22]:
view = gr.Interface(
    fn=stream_llama,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never",
    title="Llama Chat",
    description="A simple chatbot that responds in markdown."
)

view.launch()

* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


In [23]:
def stream_model(prompt, model):
    if model == "llama3.2:latest":
        result = message_llama(prompt)
    else:
        raise ValueError("Invalid model")
    yield from result

In [24]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Your message:", lines=6), gr.Dropdown(["llama3.2:latest"], label="Model:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never",
    title="Llama Chat",
    description="A simple chatbot that responds in markdown."
)

view.launch()

* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.
